In [1]:
import zipfile
import urllib.request

url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip "
file_name = "ml-latest-small.zip"

# Download the dataset
urllib.request.urlretrieve(url, file_name)


with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall(".")

print("Dataset downloaded and extracted.")


Dataset downloaded and extracted.


In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')
print(movies)
print(ratings)

data = pd.merge(ratings, movies, on='movieId')

data = data.groupby(['userId', 'title']).rating.mean().reset_index()
pivot_table = data.pivot(index='userId', columns='title', values='rating').fillna(0)

sparse_matrix = csr_matrix(pivot_table.values)

model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(sparse_matrix)


      movieId                                      title  \
0           1                           Toy Story (1995)   
1           2                             Jumanji (1995)   
2           3                    Grumpier Old Men (1995)   
3           4                   Waiting to Exhale (1995)   
4           5         Father of the Bride Part II (1995)   
...       ...                                        ...   
9737   193581  Black Butler: Book of the Atlantic (2017)   
9738   193583               No Game No Life: Zero (2017)   
9739   193585                               Flint (2017)   
9740   193587        Bungo Stray Dogs: Dead Apple (2018)   
9741   193609        Andrew Dice Clay: Dice Rules (1991)   

                                           genres  
0     Adventure|Animation|Children|Comedy|Fantasy  
1                      Adventure|Children|Fantasy  
2                                  Comedy|Romance  
3                            Comedy|Drama|Romance  
4                  

NearestNeighbors(algorithm='brute', metric='cosine')

In [3]:
def get_movie_recommendations(user_id, num_recommendations=10):
    try:
        user_idx = pivot_table.index.get_loc(user_id)
        distances, indices = model.kneighbors(sparse_matrix[user_idx], n_neighbors=num_recommendations+1)
        recommendations = [pivot_table.columns[idx] for idx in indices.flatten()]
        return recommendations[1:]
    except KeyError:
        return []


In [6]:
import tkinter as tk
from tkinter import ttk, messagebox

class MovieRecommenderApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Movie Recommender System")
        
        self.label = ttk.Label(root, text="Enter User ID:")
        self.label.grid(row=0, column=0, padx=10, pady=10)
        
        self.user_id_entry = ttk.Entry(root)
        self.user_id_entry.grid(row=0, column=1, padx=10, pady=10)
        
        self.recommend_button = ttk.Button(root, text="Get Recommendations", command=self.show_recommendations)
        self.recommend_button.grid(row=0, column=2, padx=10, pady=10)
        
        self.recommendations_listbox = tk.Listbox(root, width=50, height=15)
        self.recommendations_listbox.grid(row=1, column=0, columnspan=3, padx=10, pady=10)
        
    def show_recommendations(self):
        user_id = self.user_id_entry.get()
        if not user_id.isdigit():
            messagebox.showerror("Invalid Input", "User ID must be an integer.")
            return
        
        user_id = int(user_id)
        recommendations = get_movie_recommendations(user_id)
        
        self.recommendations_listbox.delete(0, tk.END)
        if recommendations:
            for rec in recommendations:
                self.recommendations_listbox.insert(tk.END, rec)
            print("Recommendations for user {}: {}".format(user_id, recommendations))
        else:
            self.recommendations_listbox.insert(tk.END, "No recommendations found or invalid User ID.")
            print("No recommendations found or invalid User ID for user {}".format(user_id))

if __name__ == "__main__":
    root = tk.Tk()
    app = MovieRecommenderApp(root)
    root.mainloop()


No recommendations found or invalid User ID for user 0
Recommendations for user 5: ['Analyze That (2002)', 'Admission (2013)', 'Apocalypto (2006)', 'Adventures of Buckaroo Banzai Across the 8th Dimension, The (1984)', '8 Women (2002)', 'Anchorman 2: The Legend Continues (2013)', 'Animal Crackers (1930)', '1900 (Novecento) (1976)', 'Alpha (2018)', 'Anaconda: The Offspring (2008)']
